In [5]:
import gc
import json
import pandas as pd
import numpy as np
import seaborn as sns

from chinese_calendar import is_holiday
from tqdm import tqdm_notebook
from utils import FlattenDataSimple, loadJSON, loadpkl

%matplotlib inline

In [2]:
# load csv
profiles = pd.read_csv('../input/data_set_phase1/profiles.csv')
test_plans = pd.read_csv('../input/data_set_phase1/test_plans.csv')
test_queries = pd.read_csv('../input/data_set_phase1/test_queries.csv')
train_clicks = pd.read_csv('../input/data_set_phase1/train_clicks.csv')
train_plans = pd.read_csv('../input/data_set_phase1/train_plans.csv')
train_queries = pd.read_csv('../input/data_set_phase1/train_queries.csv')

In [3]:
# merge click
train_plans = pd.merge(train_plans, train_clicks[['sid','click_mode']], on='sid', how='left')

# fill na (no click)
train_plans['click_mode'].fillna(0, inplace=True)

# set test target as nan
test_plans['click_mode'] = np.nan

# merge train & test
plans = train_plans.append(test_plans)

del train_plans, test_plans
gc.collect()

# reset index
plans.reset_index(inplace=True,drop=True)

# convert json
for key in tqdm_notebook(['distance', 'price', 'eta', 'transport_mode']):
    plans[key] = plans.plans.apply(lambda x: loadJSON(x,key))

# flatten
plans_df = [FlattenDataSimple(plans, key) for key in tqdm_notebook(['distance', 'price', 'eta', 'transport_mode'])]
plans_df = pd.concat(plans_df,axis=1)

# merge plan_time & click_mode
plans_df = pd.merge(plans_df.reset_index(), plans[['sid','plan_time', 'click_mode']], on='sid',how='outer')

# cleaning
for c in plans_df.columns.to_list():
    if 'price' in c:
        plans_df[c] = plans_df[c].replace('',0)

plans_df['plan_time'] = pd.to_datetime(plans_df['plan_time'])

In [4]:
cols_distance = ['plan_{}_distance'.format(i) for i in range(0,7)]
cols_price = ['plan_{}_price'.format(i) for i in range(0,7)]
cols_eta = ['plan_{}_eta'.format(i) for i in range(0,7)]
cols_transport_mode = ['plan_{}_transport_mode'.format(i) for i in range(0,7)]

In [5]:
plans_df.head()

,sid,plan_0_distance,plan_1_distance,plan_2_distance,plan_3_distance,plan_4_distance,plan_5_distance,plan_6_distance,plan_0_price,plan_1_price,...,plan_6_eta,plan_0_transport_mode,plan_1_transport_mode,plan_2_transport_mode,plan_3_transport_mode,plan_4_transport_mode,plan_5_transport_mode,plan_6_transport_mode,plan_time,click_mode
0,1709112,32303,33678.0,33678.0,32099.0,30446.0,NaN,NaN,600,0.0,...,NaN,9,3.0,4.0,2.0,1.0,NaN,NaN,2018-11-04 11:45:04,9.0
1,3327773,3427,3251.0,3251.0,3227.0,3227.0,NaN,NaN,300,0.0,...,NaN,2,3.0,4.0,6.0,5.0,NaN,NaN,2018-10-16 19:09:29,2.0
2,3285959,36234,29545.0,29545.0,33295.0,NaN,NaN,NaN,1100,0.0,...,NaN,7,3.0,4.0,1.0,NaN,NaN,NaN,2018-11-25 15:05:22,7.0
3,1616170,11450,9100.0,10870.0,10870.0,11722.0,NaN,NaN,400,0.0,...,NaN,2,6.0,3.0,4.0,7.0,NaN,NaN,2018-10-03 14:42:40,2.0
4,351369,18495,19111.0,19111.0,17117.0,NaN,NaN,NaN,600,0.0,...,NaN,7,3.0,4.0,1.0,NaN,NaN,NaN,2018-10-05 19:52:37,4.0


In [6]:
profiles.head()

,pid,p0,p1,p2,p3,p4,p5,p6,p7,p8,...,p56,p57,p58,p59,p60,p61,p62,p63,p64,p65
0,196356,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,204083,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,170667,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,115511,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,129719,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# merge click
train_queries = pd.merge(train_queries, train_clicks[['sid','click_mode']], on='sid', how='left')

# fill na (no click)
train_queries['click_mode'].fillna(0, inplace=True)

# set test target as nan
test_queries['click_mode'] = np.nan

# merge train & test
queries_df = train_queries.append(test_queries)

del train_queries, test_queries
gc.collect()

# to datetime
queries_df['req_time'] = pd.to_datetime(queries_df['req_time'])

# features distance
queries_df['x_o']=queries_df['o'].apply(lambda x: x.split(',')[0]).astype(float)
queries_df['y_o']=queries_df['o'].apply(lambda x: x.split(',')[1]).astype(float)
queries_df['x_d']=queries_df['d'].apply(lambda x: x.split(',')[0]).astype(float)
queries_df['y_d']=queries_df['d'].apply(lambda x: x.split(',')[1]).astype(float)

queries_df['queries_o_count']=queries_df['o'].map(queries_df['o'].value_counts())
queries_df['queries_d_count']=queries_df['d'].map(queries_df['d'].value_counts())

queries_df['queries_x_o_count']=queries_df['x_o'].map(queries_df['x_o'].value_counts())
queries_df['queries_y_o_count']=queries_df['y_o'].map(queries_df['y_o'].value_counts())
queries_df['queries_x_d_count']=queries_df['x_d'].map(queries_df['x_d'].value_counts())
queries_df['queries_y_d_count']=queries_df['y_d'].map(queries_df['y_d'].value_counts())

queries_df['queries_distance'] = np.sqrt((queries_df['x_o']-queries_df['x_d'])**2 + (queries_df['y_o']-queries_df['y_d'])**2)

queries_df['o_d'] = queries_df['o'].astype(str)+'_'+queries_df['d'].astype(str)
queries_df['queries_o_d_count'] = queries_df['o_d'].map(queries_df['o_d'].value_counts())

# datetime features
queries_df['queries_weekday'] = queries_df['req_time'].dt.weekday
queries_df['queries_hour'] = queries_df['req_time'].dt.hour
queries_df['queries_is_holiday'] = queries_df['req_time'].apply(lambda x: is_holiday(x)).astype(int)

queries_df['queries_weekday_count'] = queries_df['queries_weekday'].map(queries_df['queries_weekday'].value_counts())
queries_df['queries_hour_count'] = queries_df['queries_hour'].map(queries_df['queries_hour'].value_counts())

In [9]:
queries_df['o_d_is_holiday'] = queries_df['queries_is_holiday'].astype(str)+'_'+queries_df['o_d']
queries_df['o_d_weekday'] = queries_df['queries_weekday'].astype(str)+'_'+queries_df['o_d']
queries_df['o_d_hour'] = queries_df['queries_hour'].astype(str)+'_'+queries_df['o_d']

queries_df['o_is_holiday'] = queries_df['queries_is_holiday'].astype(str)+'_'+queries_df['o']
queries_df['o_weekday'] = queries_df['queries_weekday'].astype(str)+'_'+queries_df['o']
queries_df['o_hour'] = queries_df['queries_hour'].astype(str)+'_'+queries_df['o']

queries_df['d_is_holiday'] = queries_df['queries_is_holiday'].astype(str)+'_'+queries_df['d']
queries_df['d_weekday'] = queries_df['queries_weekday'].astype(str)+'_'+queries_df['d']
queries_df['d_hour'] = queries_df['queries_hour'].astype(str)+'_'+queries_df['d']

queries_df['queries_o_d_is_holiday_count'] = queries_df['o_d_is_holiday'].map(queries_df['o_d_is_holiday'].value_counts())
queries_df['queries_o_d_weekday_count'] = queries_df['o_d_weekday'].map(queries_df['o_d_weekday'].value_counts())
queries_df['queries_o_d_hour_count'] = queries_df['o_d_hour'].map(queries_df['o_d_hour'].value_counts())

queries_df['queries_o_is_holiday_count'] = queries_df['o_d_is_holiday'].map(queries_df['o_d_is_holiday'].value_counts())
queries_df['queries_o_weekday_count'] = queries_df['o_d_weekday'].map(queries_df['o_d_weekday'].value_counts())
queries_df['queries_o_hour_count'] = queries_df['o_d_hour'].map(queries_df['o_d_hour'].value_counts())

queries_df['queries_o_d_is_holiday_count'] = queries_df['o_d_is_holiday'].map(queries_df['o_d_is_holiday'].value_counts())
queries_df['queries_o_d_weekday_count'] = queries_df['o_d_weekday'].map(queries_df['o_d_weekday'].value_counts())
queries_df['queries_o_d_hour_count'] = queries_df['o_d_hour'].map(queries_df['o_d_hour'].value_counts())